In [9]:
torch.cuda.empty_cache()

In [1]:
# Custom functionalities
import visionNew
from Utilities.metrics import pixelwiseAccuracy
from Utilities.datasetHandler import CityscapesDataset
from Utilities.modelHandler import modelFCN

import torch
from torchvision.io.image import read_image
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import DataLoader

from torchvision.models import ResNet50_Weights
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

device = torch.device('cpu')
if torch.cuda.is_available(): device = torch.device('cuda')

In [2]:
### DATASET ###

# Read single image
dtTEST = CityscapesDataset(image_directory="TestPascal", phase=None, num_classes=21)
img = dtTEST.get_image_path("2011_003182.jpg")


# Handle real dataset
dt = CityscapesDataset(image_directory = '/mnt/data/course/psarin/inm705/pascal_voc_2012',
                       gt_directory = '/mnt/data/course/psarin/inm705/pascal_voc_2012/GT',
                       label_directory = '/mnt/data/course/psarin/inm705/pascal_voc_2012/annotations',
                       phase = 'train',
                       num_classes= 21)

# Pascal VOC categories
object_categories = ['aeroplane', 'bicycle', 'bird', 'boat',
                     'bottle', 'bus', 'car', 'cat', 'chair',
                      'cow', 'diningtable', 'dog', 'horse',
                      'motorbike', 'person', 'pottedplant',
                     'sheep', 'sofa', 'train', 'tvmonitor']
#weights.meta["categories"]


# convert list to dict
pascal_voc_classes = {}
for id, name in enumerate(object_categories): 
    pascal_voc_classes[name]=id # these are the same indices used to create the label vector y in the dataset 
    

In [3]:
## MODEL Pretrained with RESNET 50
# Initialize model with the best available weights
FCN_weights = FCN_ResNet50_Weights.DEFAULT
backbone_weights = ResNet50_Weights.DEFAULT
model = fcn_resnet50(backbone_weights=backbone_weights).to(device)
model.eval()

batch = img.unsqueeze(0).to(device)

dataloader_args = {'batch_size':100, 'shuffle':False}
testloader = DataLoader(dt, **dataloader_args) # dataloader adds the B dimension to CxHxW input image, the number of batches

In [4]:
cel = CrossEntropyLoss()
ad = Adam(model.parameters())

fcn1 = modelFCN(model, loss_function=cel, optimizer = ad )

In [10]:
L = fcn1.train_model(testloader, 1)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.37 GiB (GPU 0; 79.35 GiB total capacity; 76.27 GiB already allocated; 705.19 MiB free; 77.12 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [5]:
# Step 4: Use the model and visualize the prediction
pred = model(batch)['out']

In [5]:
idd, X, y = next(iter(testloader))

In [6]:
y.size()

torch.Size([100, 21, 350, 500])

In [7]:
X, y = X.to(device), y.to(device)
cel(model(X)['out'],y)

tensor(2.2643, device='cuda:0', grad_fn=<DivBackward1>)

torch.Size([21, 350, 500])

In [54]:
import numpy as np
n_classes = 21
g = gg.numpy()

In [65]:
test = torch.Tensor([[1,2],[3,4]])

In [66]:
test

tensor([[1., 2.],
        [3., 4.]])

tensor([[0, 0],
        [0, 0]])
tensor([[1, 0],
        [0, 0]])
tensor([[0, 1],
        [0, 0]])
tensor([[0, 0],
        [1, 0]])


In [67]:
(test==1)*1

tensor([[1, 0],
        [0, 0]])

In [57]:
g.shape

(1, 350, 500)

In [47]:
n = np.zeros((n_classes,350,500))

In [51]:
for i,nn in enumerate(n):
    pass

In [ ]:
new = torch.Tensor([])

In [38]:
gg.

tensor(1.)

In [7]:
# Step 4: Use the model and visualize the prediction
pred = model(batch)

prediction = pred["out"]
pred_aux = pred["aux"]


normalized_masks = prediction.softmax(dim=1)
normalized_aux = pred_aux.softmax(dim=1)

# Indices of classes
class_to_idx = {cls: idx for (idx, cls) in enumerate(weights.meta["categories"])}

n_lab = 2


mask = normalized_masks[0, class_to_idx["sofa"]] #+ normalized_masks[0, class_to_idx["horse"]]
to_pil_image(mask)

#print(mask.size())


KeyError: 'aux'

In [ ]:
## Import ground truth 
gt = read_image("TestPascal/2011_003182.png")


In [ ]:
pixelwiseAccuracy(prediction,gt)

In [ ]:
from bs4 import BeautifulSoup 

name = 'TestPascal/2007_000032.xml'  
with open(name, 'r') as f:
    html_string = f.read()
soup = BeautifulSoup(html_string, 'html.parser')
labels = []
area = []

while soup.object is not None:
    obj = soup.object.extract()
    obj_name = obj.find('name').get_text()
    labels.append(pascal_voc_classes.get(obj_name))


In [ ]:
labels

## Check the model

In [ ]:
from Utilities.torchsummaryNew import summary

In [ ]:
device = torch.device('cuda')

In [ ]:
x = torch.randn(1, 3, 224, 224).to(device)
model.to(device);

In [ ]:
summary(model, (3, 224, 224))